In [ ]:
import torch
import matplotlib.pyplot as plt

from train import *

import warnings
warnings.filterwarnings("ignore")

# First Order Methods
## Stochastic Gradient Descent

In [ ]:
SGD = run_experiment(optimizer_name="SGD", model_name='resnet18', 
                nb_epochs=15, batch_size=100, reduce=100, plot=True)

## Stochastic Gradient Descent + Momentum

In [ ]:
SGDM = run_experiment(optimizer_name="SGD+momentum", model_name='resnet18', 
                nb_epochs=15, batch_size=100, reduce=100, plot=True)

## Adam

In [ ]:
Adam = run_experiment(optimizer_name="adam", model_name='resnet18', 
                nb_epochs=15, batch_size=100, reduce=100, plot=True)

# Second Order Methods
## AdaHessian

In [ ]:
AdaHessian = run_experiment(optimizer_name="adahessian", model_name='resnet18', 
                nb_epochs=15, batch_size=100, reduce=100, plot=True, lr=1)

# Hybrid Method
## SGDM + AdaHessian

In [ ]:
Hybrid = run_experiment(optimizer_name="hybrid", model_name='resnet18', 
                nb_epochs=15, batch_size=100, reduce=100, plot=True, lr=0.005)

## Hyper Parameter Tuning

In [ ]:
best_return, best_hyperparameters = parameter_grid_search()